### Restart and Run All Cells

In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()
year = 2022
quarter = 3
today = date.today()
today_str = today.strftime("%Y-%m-%d")
today_str

'2023-02-14'

In [2]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit percent'.split()

format_dict = {
                'q_amt':'{:,}','q_amt_c':'{:,}','q_amt_p':'{:,}','inc_profit':'{:,}',
                'yoy_gain':'{:,}','acc_gain':'{:,}',   
                'latest_amt':'{:,}','previous_amt':'{:,}','inc_amt':'{:,}',
                'q_eps':'{:.4f}','y_eps':'{:.4f}','aq_eps':'{:.4f}','ay_eps':'{:.4f}',
                'percent':'{:.2f}%','inc_pct':'{:.2f}%'
              }

In [3]:
sql = """
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s) 
OR (year = %s-1 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC
"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2022 AND quarter <= 3) 
OR (year = 2022-1 AND quarter >= 3+1)
ORDER BY year DESC, quarter DESC



In [4]:
dfc = pd.read_sql(sql, conlt)
dfc["Counter"] = 1
dfc_grp = dfc.groupby(["name"], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp["Counter"] == 4]
dfc_grp.sample(5)

,name,year,quarter,q_amt,Counter
100,KBANK,8087,10,42479861,4
207,TOP,8087,10,37553987,4
152,RBF,8087,10,491792,4
102,KEX,8087,10,-2502428,4
10,AOT,8087,10,-14912794,4


In [5]:
sql = """
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s-1) 
OR (year = %s-1 AND quarter >= %s) 
ORDER BY year DESC, quarter DESC"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2022 AND quarter <= 3-1) 
OR (year = 2022-1 AND quarter >= 3) 
ORDER BY year DESC, quarter DESC


In [6]:
dfp = pd.read_sql(sql, conlt)
dfp["Counter"] = 1
dfp_grp = dfp.groupby(["name"], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp["Counter"] == 4]
dfp_grp.sample(5).style.format(format_dict)

,name,year,quarter,q_amt,Counter
14,ASP,8086,10,"578,771",4
58,DIF,8086,10,"12,759,649",4
8,AMATA,8086,10,"2,171,767",4
231,WICE,8086,10,"670,741",4
193,TFFIF,8086,10,"1,209,444",4


In [7]:
dfp.name.unique().shape

(233,)

In [8]:
sql = """
SELECT *
FROM stocks
"""
stocks = pd.read_sql(sql, conlt)
stocks.shape

(233, 15)

In [9]:
sql = """
SELECT *
FROM tickers
"""
tickers = pd.read_sql(sql, conlt)
tickers.shape

(401, 9)

In [10]:
dfm = pd.merge(dfc_grp, dfp_grp, on="name", suffixes=(["_c", "_p"]), how="inner")
dfm["inc_profit"] = dfm["q_amt_c"] - dfm["q_amt_p"]
dfm["percent"] = round(dfm["inc_profit"] / abs(dfm["q_amt_p"]) * 100, 2)
dfm["year"] = year
dfm["quarter"] = "Q" + str(quarter)
df_percent = dfm[cols]
df_percent.sample(5).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
185,SUPER,2022,Q3,"999,566","1,607,875","-608,309",-37.83%
93,IRPC,2022,Q3,"4,979,223","9,683,390","-4,704,167",-48.58%
139,PREB,2022,Q3,"412,854","334,580","78,274",23.39%
141,PSH,2022,Q3,"2,589,196","2,300,590","288,606",12.54%
173,SNC,2022,Q3,"726,466","795,778","-69,312",-8.71%


In [11]:
sql = """
DELETE FROM qt_profits 
WHERE year = %s AND quarter = 'Q%s'
"""
sql = sql % (year, quarter)
print(sql)

rp = conlt.execute(sql)
rp.rowcount


DELETE FROM qt_profits 
WHERE year = 2022 AND quarter = 'Q3'



221

In [12]:
sql = "SELECT name, id FROM tickers"
tickers = pd.read_sql(sql, conlt)
tickers.sample(5)

,name,id
37,BANPU,47
191,MAJOR,282
237,PRM,639
79,CPNCG,122
179,LH,267


In [13]:
df_ins = pd.merge(df_percent, tickers, on="name", how="inner")
df_ins.sample(5).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
0,ACE,2022,Q3,"1,455,601","1,485,354","-29,753",-2.00%,698
163,SCCC,2022,Q3,"3,822,686","3,895,929","-73,243",-1.88%,428
223,VIBHA,2022,Q3,"1,799,830","2,133,915","-334,085",-15.66%,610
22,BCH,2022,Q3,"5,249,486","8,548,803","-3,299,317",-38.59%,51
101,KEX,2022,Q3,"-2,502,428","-1,814,949","-687,479",-37.88%,740


In [14]:
rcds = df_ins.values.tolist()
len(rcds)

231

In [15]:
sql = """
INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, \
inc_amt, inc_pct, ticker_id) \
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""
print(sql)


INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?)



In [16]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [17]:
criteria_1 = df_ins.q_amt_c > 440_000
df_ins.loc[criteria_1, cols].sample(5).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
6,AIT,2022,Q3,"582,222","556,639","25,583",4.60%
185,SUPER,2022,Q3,"999,566","1,607,875","-608,309",-37.83%
49,CPF,2022,Q3,"18,877,729","8,395,659","10,482,070",124.85%
170,SKN,2022,Q3,"814,184","889,910","-75,726",-8.51%
109,LANNA,2022,Q3,"2,852,341","2,821,353","30,988",1.10%


In [18]:
criteria_2 = df_ins.q_amt_p > 400_000
df_ins.loc[criteria_2, cols].sample(5).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
164,SCGP,2022,Q3,"7,466,698","7,410,265","56,433",0.76%
102,KGI,2022,Q3,"1,252,420","1,307,152","-54,732",-4.19%
184,STGT,2022,Q3,"3,530,109","8,041,123","-4,511,014",-56.10%
133,OSP,2022,Q3,"2,448,105","2,784,458","-336,353",-12.08%
46,COM7,2022,Q3,"3,035,153","2,880,990","154,163",5.35%


In [19]:
criteria_3 = df_ins.percent > 10.00
df_ins.loc[criteria_3, cols].sample(5).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
35,BKI,2022,Q3,"-1,938,997","-3,805,995","1,866,998",49.05%
104,KSL,2022,Q3,"1,414,585","951,975","462,610",48.59%
86,III,2022,Q3,"492,038","446,013","46,025",10.32%
174,SPALI,2022,Q3,"8,880,962","7,851,676","1,029,286",13.11%
27,BE8,2022,Q3,"110,853","96,714","14,139",14.62%


In [20]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria, cols].sample(5).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
5,AIMIRT,2022,Q3,"693,015","566,114","126,901",22.42%
143,PTG,2022,Q3,"851,988","737,989","113,999",15.45%
146,PTTEP,2022,Q3,"65,936,017","51,309,276","14,626,741",28.51%
37,BLAND,2022,Q3,"1,215,802","1,016,832","198,970",19.57%
168,SIRI,2022,Q3,"2,831,419","2,191,557","639,862",29.20%


In [21]:
df_ins[df_ins_criteria].sort_values(by=["percent"], ascending=[False]).sample(5).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
54,CRC,2022,Q3,"6,234,646","2,819,760","3,414,886",121.11%,708
167,SINGER,2022,Q3,"954,589","858,675","95,914",11.17%,446
214,TTB,2022,Q3,"13,147,199","11,791,212","1,355,987",11.50%,541
110,LH,2022,Q3,"8,334,140","7,392,926","941,214",12.73%,267
116,M,2022,Q3,"1,517,262","851,758","665,504",78.13%,279


In [22]:
df_ins[df_ins_criteria].sort_values(by=["name"], ascending=[True]).sample(5).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
16,AWC,2022,Q3,"3,414,980","3,075,541","339,439",11.04%,699
69,FORTH,2022,Q3,"804,533","720,234","84,299",11.70%,176
148,QH,2022,Q3,"2,234,029","1,856,888","377,141",20.31%,388
214,TTB,2022,Q3,"13,147,199","11,791,212","1,355,987",11.50%,541
219,TVO,2022,Q3,"2,306,639","2,073,619","233,020",11.24%,585


In [23]:
df_ins[df_ins_criteria].sort_values(by=["name"], ascending=[True]).sample(5).style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
30,BEM,2022,Q3,"2,228,913","1,474,508","754,405",51.16%,57
174,SPALI,2022,Q3,"8,880,962","7,851,676","1,029,286",13.11%,462
116,M,2022,Q3,"1,517,262","851,758","665,504",78.13%,279
148,QH,2022,Q3,"2,234,029","1,856,888","377,141",20.31%,388
82,HFT,2022,Q3,"488,568","419,316","69,252",16.52%,207


In [24]:
sql = """
SELECT name, COUNT(*) AS 'Nbr. of Group of 4 Quarters'
FROM qt_profits
GROUP BY name
HAVING COUNT(*) = 1
ORDER BY COUNT(*) DESC, name
"""
print(sql)

tmp = pd.read_sql(sql, conlt)
tmp


SELECT name, COUNT(*) AS 'Nbr. of Group of 4 Quarters'
FROM qt_profits
GROUP BY name
HAVING COUNT(*) = 1
ORDER BY COUNT(*) DESC, name



,name,Nbr. of Group of 4 Quarters
0,BE8,1
1,ONEE,1
